Back to **[Fan](https://fanwangecon.github.io/)**'s R4Econ Homepage **[Table of Content](https://fanwangecon.github.io/R4Econ/)**

# Regression with a Variety of Outcome Variables and Right Hand Side Variables

There are M outcome variables, and there are N alternative right hand side variables. 

Regress each M outcome variable and each N alternative right hand side variable, with some common sets of controls and perhaps shared instruments. 

The output file is a M by N matrix of coefficients, with proper variable names and row names. The matrix stores coefficients for this key endogenous variable. 

Dependency: 
    - C:\Users\fan\R4Econ\linreg\ivreg\ivregdfrow.R

## Program

The program relies on double lapply. lapply is used for convenience, not speed.

In [36]:
ff_reg_mbyn <- function(list.vars.y, list.vars.x,
                        vars.c, vars.z, df, 
                        return_all = FALSE, 
                        stats_ends = 'value', time = FALSE) {
    
    # regf.iv() function is from C:\Users\fan\R4Econ\linreg\ivreg\ivregdfrow.R
    if (time) {
        start_time <- Sys.time()
    }
    
    if (return_all) {
        df.reg.out.all <- bind_rows(lapply(list.vars.x,
                              function(x) (
                                  bind_rows(lapply(list.vars.y, regf.iv, vars.x=x, vars.c=vars.c, vars.z=vars.z, df=df))
                              )))
        
    } else {
        df.reg.out.all <- (lapply(list.vars.x,
                              function(x) (
                                  bind_rows(lapply(list.vars.y, regf.iv, vars.x=x, vars.c=vars.c, vars.z=vars.z, df=df)) %>%
                                      select(vars_var.y, starts_with(x)) %>%
                                      select(vars_var.y, ends_with(stats_ends))
                              ))) %>% reduce(full_join)        
    }
     
    if (time) {
        end_time <- Sys.time()
        print(paste0('Estimation for all ys and xs took (seconds):', end_time - start_time))                                  
    }
                                  
    return(df.reg.out.all)                                  
}                          

## Load Data

In [37]:
# Library
library(tidyverse)
library(AER)

# Load Sample Data
setwd('C:/Users/fan/R4Econ/_data/')
df <- read_csv('height_weight.csv')

# Source Dependency
source('C:/Users/fan/R4Econ/linreg/ivreg/ivregdfrow.R')

# Setting
options(repr.matrix.max.rows=50, repr.matrix.max.cols=50)

Parsed with column specification:
cols(
  S.country = col_character(),
  vil.id = col_double(),
  indi.id = col_double(),
  sex = col_character(),
  svymthRound = col_double(),
  momEdu = col_double(),
  wealthIdx = col_double(),
  hgt = col_double(),
  wgt = col_double(),
  hgt0 = col_double(),
  wgt0 = col_double(),
  prot = col_double(),
  cal = col_double(),
  p.A.prot = col_double(),
  p.A.nProt = col_double()
)


## Parameters

In [38]:
var.y1 <- c('hgt')
var.y2 <- c('wgt')
var.y3 <- c('vil.id')
list.vars.y <- c(var.y1, var.y2, var.y3)

var.x1 <- c('prot')
var.x2 <- c('cal')
var.x3 <- c('wealthIdx')
var.x4 <- c('p.A.prot')
var.x5 <- c('p.A.nProt')
list.vars.x <- c(var.x1, var.x2, var.x3, var.x4, var.x5)

vars.z <- c('indi.id')
vars.c <- c('sex', 'wgt0', 'hgt0', 'svymthRound')

## Test Program OLS Z-Stat

In [39]:
vars.z <- NULL
suppressMessages(ff_reg_mbyn(list.vars.y, list.vars.x,
                             vars.c, vars.z, df, 
                             return_all = FALSE, 
                             stats_ends = 'value'))

vars_var.y,prot_tvalue,cal_tvalue,wealthIdx_tvalue,p.A.prot_tvalue,p.A.nProt_tvalue
hgt,18.8756010031786,23.4421863484661,13.508899618216,3.83682180045518,32.5448257554855
wgt,16.3591125056062,17.3686031309332,14.1390521528113,1.36958319982295,12.0961557911467
vil.id,-14.9385580468907,-19.6150110809452,34.0972558327347,8.45943342783186,17.7801422421419


## Test Program IV T-stat

In [40]:
vars.z <- c('indi.id')
suppressMessages(ff_reg_mbyn(list.vars.y, list.vars.x,
                             vars.c, vars.z, df, 
                             return_all = FALSE, 
                             stats_ends = 'value'))

vars_var.y,prot_zvalue,cal_zvalue,wealthIdx_zvalue,p.A.prot_zvalue,p.A.nProt_zvalue
hgt,8.87674929300964,12.0739764947235,4.62589553677969,26.6373587567312,32.1162192385744
wgt,5.60385871756365,6.1225187008946,5.17869536991717,11.9295584469998,12.3509307017263
vil.id,-9.22106223347162,-13.0586007975839,-51.5866689219593,-29.9627476577329,-38.3528894620707


## Test Program OLS Coefficient

In [41]:
vars.z <- NULL
suppressMessages(ff_reg_mbyn(list.vars.y, list.vars.x,
                             vars.c, vars.z, df, 
                             return_all = FALSE, 
                             stats_ends = 'Estimate'))

vars_var.y,prot_Estimate,cal_Estimate,wealthIdx_Estimate,p.A.prot_Estimate,p.A.nProt_Estimate
hgt,0.049431093806755,0.00243408846205622,0.21045655488185,3.86952250259526e-05,0.00542428867316449
wgt,16.5557424523585,0.699072500364623,106.678721085969,0.00521731297924587,0.779514232050632
vil.id,-0.0758835879205584,-0.00395676177098486,0.451733304543324,0.000149388430455142,0.00526237555581024


## Test Program IV coefficient

In [42]:
vars.z <- c('indi.id')
suppressMessages(ff_reg_mbyn(list.vars.y, list.vars.x,
                             vars.c, vars.z, df, 
                             return_all = FALSE, 
                             stats_ends = 'Estimate'))

vars_var.y,prot_Estimate,cal_Estimate,wealthIdx_Estimate,p.A.prot_Estimate,p.A.nProt_Estimate
hgt,0.859205733632614,0.0238724384575419,0.144503490136948,0.00148073028434642,0.0141317656200726
wgt,98.9428234201406,2.71948246216953,69.1816142883022,0.221916473012486,2.11856940494335
vil.id,-6.02451379136132,-0.168054407187466,-1.91414470908345,-0.00520794333267238,-0.0494468877742109


## Test Program OLS Return All

In [43]:
vars.z <- NULL
ff_reg_mbyn(list.vars.y, list.vars.x,
            vars.c, vars.z, df, 
            return_all = TRUE, 
            stats_ends = 'Estimate')

X.Intercept._Estimate,X.Intercept._Pr...t..,X.Intercept._Std.Error,X.Intercept._tvalue,adj.r.squared_v,df1_v,df2_v,df3_v,hgt0_Estimate,hgt0_Pr...t..,hgt0_Std.Error,hgt0_tvalue,prot_Estimate,prot_Pr...t..,prot_Std.Error,prot_tvalue,r.squared_v,sexMale_Estimate,sexMale_Pr...t..,sexMale_Std.Error,sexMale_tvalue,sigma_v,svymthRound_Estimate,svymthRound_Pr...t..,svymthRound_Std.Error,svymthRound_tvalue,vars_var.y,vars_vars.c,vars_vars.x,wgt0_Estimate,wgt0_Pr...t..,wgt0_Std.Error,wgt0_tvalue,cal_Estimate,cal_Pr...t..,cal_Std.Error,cal_tvalue,wealthIdx_Estimate,wealthIdx_Pr...t..,wealthIdx_Std.Error,wealthIdx_tvalue,p.A.prot_Estimate,p.A.prot_Pr...t..,p.A.prot_Std.Error,p.A.prot_tvalue,p.A.nProt_Estimate,p.A.nProt_Pr...t..,p.A.nProt_Std.Error,p.A.nProt_tvalue
27.3528514188608,5.68247182214952e-231,0.831272666092284,32.9047886867776,0.814249026159781,6,18957,6,0.60391817340617,1.14533314566771e-183,0.0206657538633713,29.2231378249683,0.049431093806755,9.54769322304645e-79,0.00261878251179557,18.8756010031786,0.814298005954592,0.935177182449406,2.36432111724607e-51,0.0618482294097262,15.1205166481668,4.21029844914315,0.87166589100565,0,0.00387681209575621,224.840892330022,hgt,sex+wgt0+hgt0+svymthRound,prot,-0.000146104685986986,0.136011583497549,9.79994437486573e-05,-1.49087260496811,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
99.873884728925,0.75529705553815,320.450650378664,0.31166697465244,0.60716936506893,6,18962,6,56.3852027199184,1.52417506966835e-12,7.96735224000553,7.0770314931977,16.5557424523585,9.61203373222183e-60,1.01201959743751,16.3591125056062,0.607272921412825,415.163616765357,2.48252880290814e-67,23.8518341439675,17.4059409544552,1623.77111076428,189.04290688382,0,1.4955473831309,126.403823119306,wgt,sex+wgt0+hgt0+svymthRound,prot,0.637023553461055,2.96480083692757e-63,0.0378027371614794,16.8512547316329,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
31.4646660224049,6.78164655340399e-84,1.61328519718754,19.503474077155,0.0373247512680971,6,18999,6,-0.296844389234445,1.40290395213743e-13,0.0401060913799595,-7.40147890309685,-0.0758835879205584,3.56396093562335e-50,0.00507971302734622,-14.9385580468907,0.0375780335372857,-0.254089999175318,0.0343768259467621,0.120093045309631,-2.11577613441484,8.18491760066961,-0.0154759587993917,0.0397984032097113,0.00752730297891317,-2.05597660181154,vil.id,sex+wgt0+hgt0+svymthRound,prot,-0.000903390591533867,2.05763549729273e-06,0.000190221503167431,-4.74915073475531,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
27.9038445914729,8.24252673989353e-242,0.828072565159449,33.6973421962119,0.81608922805658,6,18957,6,0.589847843438394,7.79174951119325e-177,0.0205836398278421,28.6561486875877,NA,NA,NA,NA,0.816137722617266,0.893484662055608,2.08765935335877e-47,0.0616078355613525,14.5027763743757,4.18939119979502,0.851989049736817,0,0.00411253488213795,207.168832400006,hgt,sex+wgt0+hgt0+svymthRound,cal,-0.000116898230009949,0.230228828649018,9.74307633896921e-05,-1.19980821193398,0.00243408846205622,8.01672708877986e-120,0.000103833679413418,23.4421863484661,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
219.626705179399,0.493216914827181,320.522532223672,0.685214557790078,0.607863678511207,6,18962,6,52.9707041800704,3.05720143843395e-11,7.96822145797115,6.64774497790599,NA,NA,NA,NA,0.60796705182314,405.534891838028,2.51355675686752e-64,23.8567507583516,16.9987478993157,1622.33549880859,185.318286001897,0,1.59266949679221,116.357025971267,wgt,sex+wgt0+hgt0+svymthRound,cal,0.649394003614758,7.43034302413852e-66,0.037739875283113,17.2071051836606,0.699072500364623,4.71331900885298e-67,0.0402492068645167,17.3686031309332,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
30.5103987898551,1.62608789535248e-79,1.60831193651104,18.9704485163756,0.0453498711076042,6,18999,6,-0.273219210757899,8.49149153665126e-12,0.0399777363511633,-6.83428417151858,NA,NA,NA,NA,0.0456010419476623,-0.181389489610951,0.129768754080748,0.11972270545355,-1.51508010885476,8.15073036560541,0.0201471237605442,0.0117151185126433,0.007992178

## Test Program IV Return All

In [44]:
vars.z <- c('indi.id')
ff_reg_mbyn(list.vars.y, list.vars.x,
            vars.c, vars.z, df, 
            return_all = TRUE, 
            stats_ends = 'Estimate')

X.Intercept._Estimate,X.Intercept._Pr...z..,X.Intercept._Std.Error,X.Intercept._zvalue,hgt0_Estimate,hgt0_Pr...z..,hgt0_Std.Error,hgt0_zvalue,prot_Estimate,prot_Pr...z..,prot_Std.Error,prot_zvalue,Sargan_df1,sexMale_Estimate,sexMale_Pr...z..,sexMale_Std.Error,sexMale_zvalue,svymthRound_Estimate,svymthRound_Pr...z..,svymthRound_Std.Error,svymthRound_zvalue,vars_var.y,vars_vars.c,vars_vars.x,vars_vars.z,...,Weakinstruments_statistic,wgt0_Estimate,wgt0_Pr...z..,wgt0_Std.Error,wgt0_zvalue,Wu.Hausman_df1,Wu.Hausman_df2,Wu.Hausman_p.value,Wu.Hausman_statistic,cal_Estimate,cal_Pr...z..,cal_Std.Error,cal_zvalue,wealthIdx_Estimate,wealthIdx_Pr...z..,wealthIdx_Std.Error,wealthIdx_zvalue,p.A.prot_Estimate,p.A.prot_Pr...z..,p.A.prot_Std.Error,p.A.prot_zvalue,p.A.nProt_Estimate,p.A.nProt_Pr...z..,p.A.nProt_Std.Error,p.A.nProt_zvalue
40.2173991882938,3.69748206920405e-59,2.47963650430699,16.2190704639323,0.403139725681418,1.25009876641748e-13,0.0543948312973965,7.41136089709158,0.859205733632614,6.88427338202428e-19,0.0967928354481331,8.87674929300964,0,0.154043421788007,0.38807812932888,0.178475271469781,0.86310792817082,0.20990165085783,0.00846239710392287,0.0797183179471441,2.63304164291327,hgt,sex+wgt0+hgt0+svymthRound,prot,indi.id,...,82.0931934821266,-0.00163274724538111,4.88365163639597e-08,0.00029928487659495,-5.45549532591606,1,18956,1.53929570343279e-118,543.467268879953,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1408.1626637032,0.00217397545504963,459.377029874119,3.06537456626657,35.5765914326678,0.000445802636381424,10.1318250572006,3.51137048180512,98.9428234201406,2.09631602352917e-08,17.6561952052848,5.60385871756365,0,333.799680049259,5.06413216642981e-24,33.0216035385405,10.1085242471545,121.78985943172,5.96047652813855e-17,14.5577085129475,8.36600480930094,wgt,sex+wgt0+hgt0+svymthRound,prot,indi.id,...,79.8251182827386,0.492582112313709,2.33136555228405e-20,0.0532753838702833,9.24596082710666,1,18961,3.13415891402799e-08,30.6481856102772,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
-64.490636067872,0.000109756271656929,16.673099250727,-3.86794531107106,1.20995060148712,0.00097112649404847,0.366789440587685,3.29876072644971,-6.02451379136132,2.94171378745816e-20,0.653342710289155,-9.22106223347162,0,5.41175429817609,5.80077629932476e-06,1.19371921154418,4.53352366774387,4.84745570027424,2.07373887977152e-19,0.538050140685815,9.00930105527994,vil.id,sex+wgt0+hgt0+svymthRound,prot,indi.id,...,83.8989817367586,0.00999798623641602,7.95432753711715e-07,0.00202532507408065,4.93648469787221,1,18998,0,5652.51924792859,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
39.6732302990235,1.30030240177373e-103,1.83545587849039,21.6149190857443,0.357976348180876,2.82141265004339e-17,0.0423453726223874,8.45373003027063,NA,NA,NA,NA,0,0.106307556057668,0.423490075745117,0.132821186086547,0.800381017440976,0.322893837128574,9.66146445882893e-11,0.0498896912188091,6.47215545416802,hgt,sex+wgt0+hgt0+svymthRound,cal,indi.id,...,164.392129625299,-0.000658938519302931,0.00032843149807424,0.000183457551985601,-3.59177647456371,1,18956,2.88592507054107e-108,494.955883488045,0.0238724384575419,1.44956616452661e-33,0.00197718112735887,12.0739764947235,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1325.54736576331,0.00138952700443324,414.645900526211,3.19681772828602,31.0172706497394,0.0013100303315764,9.65135595900306,3.21377335801252,NA,NA,NA,NA,0,330.452608866758,2.52735690930834e-27,30.5174257711927,10.8283251459136,135.494858749214,4.48931446042076e-34,11.133488331472,12.1700274626596,wgt,sex+wgt0+hgt0+svymthRound,cal,indi.id,...,162.747072038429,0.601258436431587,2.0921134733036e-48,0.0411255751282477,14.6200614716414,1,18961,7.6495944085204e-07,24.4605456760994,2.71948246216953,9.21076021290446e-10,0.444177077282291,6.1225187008946,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
-59.8304089440729,3.75547414421179e-07,11.7754321198995,-5.08095230263053,1.5037447089682,3.70002169470828e-08,0.273179527952317,5.50460248701607,NA,NA,NA,NA,0,5.83118942788808,6.1228

## Test Line by Line

### Set Up Parameters

In [45]:
vars.z <- c('indi.id')
vars.z <- NULL
vars.c <- c('sex', 'wgt0', 'hgt0', 'svymthRound')

### First Lapply

In [46]:
df.reg.out <- as_tibble(bind_rows(lapply(list.vars.y, regf.iv, vars.x=var.x1, vars.c=vars.c, vars.z=vars.z, df=df)))

### Nested Lapply Test

In [47]:
lapply(list.vars.y, function(y) (mean(df[[var.x1]], na.rm=TRUE) + mean(df[[y]], na.rm=TRUE)))
lapplytwice <- lapply(list.vars.x, function(x) (lapply(list.vars.y, function(y) (mean(df[[x]], na.rm=TRUE) + mean(df[[y]], na.rm=TRUE)))))
lapplytwice

[[1]]
[1] 98.3272

[[2]]
[1] 13626.51

[[3]]
[1] 26.11226

[[1]]
[[1]][[1]]
[1] 98.3272

[[1]][[2]]
[1] 13626.51

[[1]][[3]]
[1] 26.11226


[[2]]
[[2]][[1]]
[1] 525.4708

[[2]][[2]]
[1] 14053.65

[[2]][[3]]
[1] 453.2558


[[3]]
[[3]][[1]]
[1] 90.69287

[[3]][[2]]
[1] 13618.87

[[3]][[3]]
[1] 18.47793


[[4]]
[[4]][[1]]
[1] 2095.3

[[4]][[2]]
[1] 15623.48

[[4]][[3]]
[1] 2023.085


[[5]]
[[5]][[1]]
[1] 271.2886

[[5]][[2]]
[1] 13799.47

[[5]][[3]]
[1] 199.0737

### Nested Lapply All

In [48]:
df.reg.out.all <- bind_rows(lapply(list.vars.x,
                      function(x) (
                          bind_rows(lapply(list.vars.y, regf.iv, vars.x=x, vars.c=vars.c, vars.z=vars.z, df=df)) 
                      )))

In [49]:
df.reg.out.all

X.Intercept._Estimate,X.Intercept._Pr...t..,X.Intercept._Std.Error,X.Intercept._tvalue,adj.r.squared_v,df1_v,df2_v,df3_v,hgt0_Estimate,hgt0_Pr...t..,hgt0_Std.Error,hgt0_tvalue,prot_Estimate,prot_Pr...t..,prot_Std.Error,prot_tvalue,r.squared_v,sexMale_Estimate,sexMale_Pr...t..,sexMale_Std.Error,sexMale_tvalue,sigma_v,svymthRound_Estimate,svymthRound_Pr...t..,svymthRound_Std.Error,svymthRound_tvalue,vars_var.y,vars_vars.c,vars_vars.x,wgt0_Estimate,wgt0_Pr...t..,wgt0_Std.Error,wgt0_tvalue,cal_Estimate,cal_Pr...t..,cal_Std.Error,cal_tvalue,wealthIdx_Estimate,wealthIdx_Pr...t..,wealthIdx_Std.Error,wealthIdx_tvalue,p.A.prot_Estimate,p.A.prot_Pr...t..,p.A.prot_Std.Error,p.A.prot_tvalue,p.A.nProt_Estimate,p.A.nProt_Pr...t..,p.A.nProt_Std.Error,p.A.nProt_tvalue
27.3528514188608,5.68247182214952e-231,0.831272666092284,32.9047886867776,0.814249026159781,6,18957,6,0.60391817340617,1.14533314566771e-183,0.0206657538633713,29.2231378249683,0.049431093806755,9.54769322304645e-79,0.00261878251179557,18.8756010031786,0.814298005954592,0.935177182449406,2.36432111724607e-51,0.0618482294097262,15.1205166481668,4.21029844914315,0.87166589100565,0,0.00387681209575621,224.840892330022,hgt,sex+wgt0+hgt0+svymthRound,prot,-0.000146104685986986,0.136011583497549,9.79994437486573e-05,-1.49087260496811,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
99.873884728925,0.75529705553815,320.450650378664,0.31166697465244,0.60716936506893,6,18962,6,56.3852027199184,1.52417506966835e-12,7.96735224000553,7.0770314931977,16.5557424523585,9.61203373222183e-60,1.01201959743751,16.3591125056062,0.607272921412825,415.163616765357,2.48252880290814e-67,23.8518341439675,17.4059409544552,1623.77111076428,189.04290688382,0,1.4955473831309,126.403823119306,wgt,sex+wgt0+hgt0+svymthRound,prot,0.637023553461055,2.96480083692757e-63,0.0378027371614794,16.8512547316329,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
31.4646660224049,6.78164655340399e-84,1.61328519718754,19.503474077155,0.0373247512680971,6,18999,6,-0.296844389234445,1.40290395213743e-13,0.0401060913799595,-7.40147890309685,-0.0758835879205584,3.56396093562335e-50,0.00507971302734622,-14.9385580468907,0.0375780335372857,-0.254089999175318,0.0343768259467621,0.120093045309631,-2.11577613441484,8.18491760066961,-0.0154759587993917,0.0397984032097113,0.00752730297891317,-2.05597660181154,vil.id,sex+wgt0+hgt0+svymthRound,prot,-0.000903390591533867,2.05763549729273e-06,0.000190221503167431,-4.74915073475531,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
27.9038445914729,8.24252673989353e-242,0.828072565159449,33.6973421962119,0.81608922805658,6,18957,6,0.589847843438394,7.79174951119325e-177,0.0205836398278421,28.6561486875877,NA,NA,NA,NA,0.816137722617266,0.893484662055608,2.08765935335877e-47,0.0616078355613525,14.5027763743757,4.18939119979502,0.851989049736817,0,0.00411253488213795,207.168832400006,hgt,sex+wgt0+hgt0+svymthRound,cal,-0.000116898230009949,0.230228828649018,9.74307633896921e-05,-1.19980821193398,0.00243408846205622,8.01672708877986e-120,0.000103833679413418,23.4421863484661,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
219.626705179399,0.493216914827181,320.522532223672,0.685214557790078,0.607863678511207,6,18962,6,52.9707041800704,3.05720143843395e-11,7.96822145797115,6.64774497790599,NA,NA,NA,NA,0.60796705182314,405.534891838028,2.51355675686752e-64,23.8567507583516,16.9987478993157,1622.33549880859,185.318286001897,0,1.59266949679221,116.357025971267,wgt,sex+wgt0+hgt0+svymthRound,cal,0.649394003614758,7.43034302413852e-66,0.037739875283113,17.2071051836606,0.699072500364623,4.71331900885298e-67,0.0402492068645167,17.3686031309332,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
30.5103987898551,1.62608789535248e-79,1.60831193651104,18.9704485163756,0.0453498711076042,6,18999,6,-0.273219210757899,8.49149153665126e-12,0.0399777363511633,-6.83428417151858,NA,NA,NA,NA,0.0456010419476623,-0.181389489610951,0.129768754080748,0.11972270545355,-1.51508010885476,8.15073036560541,0.0201471237605442,0.0117151185126433,0.007992178

### Nested Lapply Select

In [50]:
df.reg.out.all <- (lapply(list.vars.x,
                      function(x) (
                          bind_rows(lapply(list.vars.y, regf.iv, vars.x=x, vars.c=vars.c, vars.z=vars.z, df=df)) %>%
                              select(vars_var.y, starts_with(x)) %>%
                              select(vars_var.y, ends_with('value'))
                      ))) %>% reduce(full_join)

Joining, by = "vars_var.y"
Joining, by = "vars_var.y"
Joining, by = "vars_var.y"
Joining, by = "vars_var.y"


In [51]:
df.reg.out.all

vars_var.y,prot_tvalue,cal_tvalue,wealthIdx_tvalue,p.A.prot_tvalue,p.A.nProt_tvalue
hgt,18.8756010031786,23.4421863484661,13.508899618216,3.83682180045518,32.5448257554855
wgt,16.3591125056062,17.3686031309332,14.1390521528113,1.36958319982295,12.0961557911467
vil.id,-14.9385580468907,-19.6150110809452,34.0972558327347,8.45943342783186,17.7801422421419
